In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

var_BS = "BS"
var_CF = "CF"
var_IS = "IS"

folder_combined = <>
folder_pas = <>
folder_cos = <>
folder_mai = <>


In [ ]:
df_all_10K = pd.DataFrame()
# Get all Entries
df_col_types = {"sic":str,"cik":str,"fy":str,"qtrs":str}
df_all_10K = pd.read_csv((folder_pas+"10-K_AllEntries.csv.zip"),dtype=df_col_types)
df_all_10K.drop(["Unnamed: 0"],axis=1,inplace=True)
df_all_10K["tag"] = df_all_10K["tag"].str.upper()
print(len(df_all_10K))

df_col_types = {"sic":str,"cik":str,"fy":str,"qtrs":str}
df_all_20F = pd.read_csv((folder_pas+"20-F_AllEntries.csv.zip"),dtype=df_col_types)
df_all_20F.drop(["Unnamed: 0"],axis=1,inplace=True)
df_all_20F["tag"] = df_all_20F["tag"].str.upper()
print(len(df_all_20F))

df_col_types = {"sic":str,"cik":str,"fy":str,"qtrs":str}
df_all_40F = pd.read_csv((folder_pas+"40-F_AllEntries.csv.zip"),dtype=df_col_types)
df_all_40F.drop(["Unnamed: 0"],axis=1,inplace=True)
df_all_40F["tag"] = df_all_40F["tag"].str.upper()
print(len(df_all_40F))

df_ANNUAL = df_all_10K.copy()
print(len(df_ANNUAL))
df_ANNUAL = df_ANNUAL.append(df_all_20F,ignore_index=True)
print(len(df_ANNUAL))
df_ANNUAL = df_ANNUAL.append(df_all_40F,ignore_index=True)
print(len(df_ANNUAL))

df_ANNUAL_temp = pd.DataFrame()
df_all_10K = pd.DataFrame()
df_all_20F = pd.DataFrame()
df_all_40F = pd.DataFrame()

df_ANNUAL["date_period"] = pd.to_datetime(df_ANNUAL["date_period"])
df_ANNUAL = df_ANNUAL[(~(df_ANNUAL["fy"].isnull()))]
df_ANNUAL["calyr"] = df_ANNUAL["date_period"].dt.year
df_ANNUAL["ffy"] = np.where(df_ANNUAL["fy"].astype(int) <= df_ANNUAL["calyr"],df_ANNUAL["calyr"],df_ANNUAL["fy"])
df_ANNUAL["cfy"] = df_ANNUAL["cik"] + "-" + df_ANNUAL["ffy"].astype(str)

df_currency = pd.read_excel((folder_mai+"Currency Data.xlsx"),sheet_name="subset")

# GET ALL TAGS
df_all_tags = pd.read_excel((folder_mai+"Tags and Stats.xlsx"),sheet_name="Tag List")
df_all_tags.sort_values(by=["stmt","metric","tag_order"],ascending=True,inplace=True)
print(len(df_all_tags))

# GET THE INDUSTRY AND THE SECTOR
df_col_types = {"sic":str,"cik":str,"fy":str,"qtrs":str}
df_cik_sic_align = pd.read_csv((folder_mai+"cik_sic_align.csv"),dtype=df_col_types)
df_sic_data = df_cik_sic_align[["sic","Office","Industry Title"]] 
df_sic_data.drop_duplicates(subset="sic",inplace=True)

df_ANNUAL.to_csv((folder_pas+" ALL ANNUAL ENTRIES.csv"))
print(len(df_ANNUAL))
df_ANNUAL = df_ANNUAL[["cik","name","sic","countryba","stmt","tag","negating","date_period","qtrs","uom","value","ffy","cfy"]][df_ANNUAL["coreg"].isnull()]
print(len(df_ANNUAL))



In [ ]:
df_col_types = {"sic":str,"cik":str,"fy":str,"qtrs":str,"calyr":str,"ffy":str}
df_ANNUAL = pd.read_csv((folder_pas+" ALL ANNUAL ENTRIES.csv.zip"),dtype=df_col_types)
print(len(df_ANNUAL))
df_ANNUAL = df_ANNUAL[["cik","name","sic","countryba","stmt","tag","negating","date_period","qtrs","uom","value","ffy","cfy"]][df_ANNUAL["coreg"].isnull()]
print(len(df_ANNUAL))

# GET ALL TAGS
df_all_tags = pd.read_excel((folder_mai+"Tags and Stats.xlsx"),sheet_name="Tag List")
df_all_tags.sort_values(by=["stmt","metric","tag_order"],ascending=True,inplace=True)
print(len(df_all_tags))

# GET THE INDUSTRY AND THE SECTOR
df_col_types = {"sic":str,"cik":str,"fy":str,"qtrs":str}
df_cik_sic_align = pd.read_csv((folder_mai+"cik_sic_align.csv"),dtype=df_col_types)
df_sic_data = df_cik_sic_align[["sic","Office","Industry Title"]] 
df_sic_data.drop_duplicates(subset="sic",inplace=True)

df_currency = pd.read_excel((folder_mai+"Currency Data.xlsx"),sheet_name="subset")



# Data of a Single Company

In [ ]:
var_co_name = "SALES"
df_ANNUAL_temp = df_ANNUAL.copy()
df_co_data = df_ANNUAL_temp[df_ANNUAL_temp["name"].str.contains(var_co_name)]
df_ANNUAL_temp = pd.DataFrame()
df_co_data.drop_duplicates(subset=["name"],inplace=True)
df_co_data


In [ ]:
# Get the data of the Foreign company 
var_co_name = "SFDC"
var_cik = "1108524"
df_ANNUAL_temp = df_ANNUAL.copy()
df_co = df_ANNUAL_temp[df_ANNUAL_temp["cik"].str.contains(var_cik)]
df_ANNUAL_temp = pd.DataFrame()
print(len(df_co))
df_co.to_csv((folder_cos+var_co_name+" "+var_cik+" ANNUAL.csv"))

# Income Statement Entries

In [ ]:
var_FS = var_IS

# GET METRICs 
df_met_tags = df_all_tags["metric"][(df_all_tags["stmt"] == var_FS)]
df_met_tags.drop_duplicates(inplace=True)
df_met_tags = pd.DataFrame(df_met_tags)

df_met_tags2 = df_met_tags.reset_index(level=0,drop=False)
df_met_tags2.drop("index",axis=1,inplace=True)
df_met_tags4 = df_met_tags2["metric"]

df_all_tags_copy = df_all_tags.copy()
counter_met = 0

while counter_met < len(df_met_tags4):
    
# GET RELEVANT TAGS    
    df_rel_tags = df_all_tags_copy[(df_all_tags_copy["metric"] == df_met_tags4[counter_met])&((df_all_tags_copy["stmt"] == var_FS))]
    df_rel_tags.drop_duplicates(inplace=True)
    df_rel_tags = pd.DataFrame(df_rel_tags)
    print(("Tags in Metric "+df_met_tags4[counter_met]+" are "+np.str(len(df_rel_tags))))

# PARE DOWN THE FINANCIAL STATEMENT DATA TO ONLY THOSE TAGS THAT ARE RELEVANT

    df_ANNUAL_temp = df_ANNUAL.copy()
    df_ANNUAL_rel = df_ANNUAL_temp[(df_ANNUAL_temp["stmt"] == var_FS) 
                                   &(df_ANNUAL_temp["tag"].isin(df_rel_tags["tag"]))
                               &(df_ANNUAL_temp["qtrs"] == "4")]

    print(("Entries with "+df_met_tags4[counter_met]+" relevant tags are "+np.str(len(df_ANNUAL_rel))))
    
    # RESET THE INDEX ON THE TAG DATAFRAME (# see INDEX_WOES notes)
    df_rel_tags2 = df_rel_tags.reset_index(level=0,drop=False)
    df_rel_tags2.drop("index",axis=1,inplace=True)
    df_rel_tags4 = df_rel_tags2["tag"]
    
    df_ANNUAL_temp = df_ANNUAL_rel.copy()
    df_rel_cos = df_ANNUAL_temp[["cfy"]]
    df_rel_cos.drop_duplicates(subset=["cfy"],inplace=True)
    df_ANNUAL_temp = pd.DataFrame()
    df_rel_cos_temp = df_rel_cos.copy()
    print(("Number of Relevant Co-FY Combos "+np.str(len(df_rel_cos_temp))))
    
    # Create a copy of the relevant entries
    df_ANNUAL_rcop = df_ANNUAL_rel.copy()

    df_entries_temp = pd.DataFrame()
    df_cosfy_tagfound = pd.DataFrame()
    df_cosfy_alltagfound = pd.DataFrame()
    df_cf_list = pd.DataFrame()
    counter_tag = 0
    
    while counter_tag < len(df_rel_tags4):
    
        df_entries_temp = df_ANNUAL_rcop[df_ANNUAL_rcop["cfy"].isin(df_rel_cos_temp["cfy"])]
        df_cosfy_tagfound = df_entries_temp[(df_entries_temp["tag"] == df_rel_tags4[counter_tag])]
    
        df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)
    
        df_cf_list = df_cosfy_tagfound[["cfy"]]
        df_cf_list.drop_duplicates()
#   df_cf_list["cik-fy"] = df_cf_list["cik"] + "-" + df_cf_list["fy"]    
        df_rel_cos_temp = df_rel_cos_temp[~(df_rel_cos_temp["cfy"].isin(df_cf_list["cfy"]))]
    
        print((df_rel_tags4[counter_tag]+ " entries "+np.str(len(df_cosfy_tagfound))))
    
        counter_tag = counter_tag + 1

    df_cosfy_alltagfound = pd.merge(df_cosfy_alltagfound,df_currency,left_on="uom",right_on="Currency",how="left")
    df_cosfy_alltagfound["value Bn USD"] = df_cosfy_alltagfound["value"] / df_cosfy_alltagfound["inUSD"] / 1000000000

    # MERGE THE RELEVANT ENTRIES WITH THE INDUSTRY AND SECTOR
    df_10K_met_found = pd.merge(df_cosfy_alltagfound,df_sic_data,on="sic",how="left")
    
    print("Entries Found "+np.str(len(df_10K_met_found)))
    df_10K_met_found.to_csv((folder_cos+"ANNUAL "+var_FS+" "+df_met_tags4[counter_met]+" 1 FOUND.csv"))

    df_10K_fnd_cik_fy = df_10K_met_found["cfy"]
    df_10K_fnd_cik_fy.drop_duplicates(inplace=True)
    df_10K_fnd_cik_fy = pd.DataFrame(df_10K_fnd_cik_fy)
    print("CIK-FY Tag Found "+np.str(len(df_10K_fnd_cik_fy)))

    df_10K_all_cik_fy = df_ANNUAL["cfy"]
    df_10K_all_cik_fy.drop_duplicates(inplace=True)
    df_10K_all_cik_fy = pd.DataFrame(df_10K_all_cik_fy)
    print("CIK-FY All "+np.str(len(df_10K_all_cik_fy)))

    df_10K_cik_missing = df_10K_all_cik_fy[~(df_10K_all_cik_fy["cfy"].isin(df_10K_fnd_cik_fy["cfy"]))]
    print("CIK-FY Missing "+np.str(len(df_10K_cik_missing)))
    
    # GET THE IRRELEVANT TAGS
    df_irl_tags = df_all_tags["tag"][~(df_all_tags["metric"] == df_met_tags4[counter_met])&(df_all_tags["stmt"] == var_FS)]
    df_irl_tags.drop_duplicates(inplace=True)
    df_irl_tags = pd.DataFrame(df_irl_tags)

# GET THE GENERIC TAGS
    df_gnr_tags = df_all_tags["tag_generic"][(df_all_tags["metric"] == df_met_tags4[counter_met])&((df_all_tags["stmt"] == var_FS))]
    df_gnr_tags.drop_duplicates(inplace=True)
    df_gnr_tags = pd.DataFrame(df_gnr_tags)
    df_gnr_tags.rename(columns={"tag_generic":"tag"},inplace=True)

# Found Generic Tag
    df_all_10K_src = df_ANNUAL.copy()
#df_found_generic = df_all_10K[(df_10K_copy["stmt"] == var_IS)&(df_10K_copy["form"] == "10-K")&(df_10K_copy["qtrs"] == "4")&(df_all_10K["cik-fy"].isin(df_10K_cik_missing["cik-fy"]))&(df_all_10K["tag"].str.contains("|".join(df_gnr_tags["tag"])))]
    df_found_generic = df_all_10K_src[(df_all_10K_src["stmt"] == var_FS)
                                  &(df_all_10K_src["qtrs"] == "4")&(df_all_10K_src["cfy"].isin(df_10K_cik_missing["cfy"]))
                                  &(df_all_10K_src["tag"].str.contains("|".join(df_gnr_tags["tag"])))]
#df_found_generic.to_csv((folder_cos+"10K"+var_rel_met+" 2 GENERIC FND.csv"))
#df_10K_fnd_gnr = df_found_generic["cik-fy"]
#df_10K_fnd_gnr.drop_duplicates(inplace=True)
    df_found_generic.drop_duplicates(subset=["tag"],inplace=True)
    print("Found Generic "+np.str(len(df_found_generic)))
    df_found_generic.to_csv((folder_cos+"ANNUAL "+var_FS+" "+df_met_tags4[counter_met]+" 2 GENERIC FND.csv"))
    
    counter_met = counter_met + 1

In [ ]:
df_irl_tags

In [ ]:
df_rel_tags

# Balance Sheet Entries

In [ ]:
var_FS = var_BS

# GET METRICs 
df_met_tags = df_all_tags["metric"][(df_all_tags["stmt"] == var_FS)]
df_met_tags.drop_duplicates(inplace=True)
df_met_tags = pd.DataFrame(df_met_tags)

df_met_tags2 = df_met_tags.reset_index(level=0,drop=False)
df_met_tags2.drop("index",axis=1,inplace=True)
df_met_tags4 = df_met_tags2["metric"]

df_all_tags_copy = df_all_tags.copy()
counter_met = 0

while counter_met < len(df_met_tags4):
    
# GET RELEVANT TAGS    
    df_rel_tags = df_all_tags_copy[(df_all_tags_copy["metric"] == df_met_tags4[counter_met])&((df_all_tags_copy["stmt"] == var_FS))]
    df_rel_tags.drop_duplicates(inplace=True)
    df_rel_tags = pd.DataFrame(df_rel_tags)
    print(("Tags in Metric "+df_met_tags4[counter_met]+" are "+np.str(len(df_rel_tags))))

# PARE DOWN THE FINANCIAL STATEMENT DATA TO ONLY THOSE TAGS THAT ARE RELEVANT
    df_ANNUAL_temp = df_ANNUAL.copy()
    df_ANNUAL_rel = df_ANNUAL_temp[(df_ANNUAL_temp["stmt"] == var_FS) &(df_ANNUAL_temp["tag"].isin(df_rel_tags["tag"]))
                               ]

    print(("Entries with "+df_met_tags4[counter_met]+" relevant tags are "+np.str(len(df_ANNUAL_rel))))
    
    # RESET THE INDEX ON THE TAG DATAFRAME (# see INDEX_WOES notes)
    df_rel_tags2 = df_rel_tags.reset_index(level=0,drop=False)
    df_rel_tags2.drop("index",axis=1,inplace=True)
    df_rel_tags4 = df_rel_tags2["tag"]
    
    df_ANNUAL_temp = df_ANNUAL_rel.copy()
    df_rel_cos = df_ANNUAL_temp[["cfy"]]
    df_rel_cos.drop_duplicates(subset=["cfy"],inplace=True)
    df_ANNUAL_temp = pd.DataFrame()
    df_rel_cos_temp = df_rel_cos.copy()
    print(("Number of Relevant Co-FY Combos "+np.str(len(df_rel_cos_temp))))
    
    # Create a copy of the relevant entries
    df_ANNUAL_rcop = df_ANNUAL_rel.copy()

    df_entries_temp = pd.DataFrame()
    df_cosfy_tagfound = pd.DataFrame()
    df_cosfy_alltagfound = pd.DataFrame()
    df_cf_list = pd.DataFrame()
    counter_tag = 0
    
    while counter_tag < len(df_rel_tags4):
    
        df_entries_temp = df_ANNUAL_rcop[df_ANNUAL_rcop["cfy"].isin(df_rel_cos_temp["cfy"])]
        df_cosfy_tagfound = df_entries_temp[(df_entries_temp["tag"] == df_rel_tags4[counter_tag])]
    
        df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)
    
        df_cf_list = df_cosfy_tagfound[["cfy"]]
        df_cf_list.drop_duplicates()
#   df_cf_list["cik-fy"] = df_cf_list["cik"] + "-" + df_cf_list["fy"]    
        df_rel_cos_temp = df_rel_cos_temp[~(df_rel_cos_temp["cfy"].isin(df_cf_list["cfy"]))]
    
        print((df_rel_tags4[counter_tag]+ " entries "+np.str(len(df_cosfy_tagfound))))
    
        counter_tag = counter_tag + 1

    df_cosfy_alltagfound = pd.merge(df_cosfy_alltagfound,df_currency,left_on="uom",right_on="Currency",how="left")
    df_cosfy_alltagfound["value Bn USD"] = df_cosfy_alltagfound["value"] / df_cosfy_alltagfound["inUSD"] / 1000000000

    # MERGE THE RELEVANT ENTRIES WITH THE INDUSTRY AND SECTOR
    df_10K_met_found = pd.merge(df_cosfy_alltagfound,df_sic_data,on="sic",how="left")
    
    print("Entries Found "+np.str(len(df_10K_met_found)))
    df_10K_met_found.to_csv((folder_cos+"ANNUAL "+var_FS+" "+df_met_tags4[counter_met]+" 1 FOUND.csv"))

    df_10K_fnd_cik_fy = df_10K_met_found["cfy"]
    df_10K_fnd_cik_fy.drop_duplicates(inplace=True)
    df_10K_fnd_cik_fy = pd.DataFrame(df_10K_fnd_cik_fy)
    print("CIK-FY Tag Found "+np.str(len(df_10K_fnd_cik_fy)))

    df_10K_all_cik_fy = df_ANNUAL["cfy"]
    df_10K_all_cik_fy.drop_duplicates(inplace=True)
    df_10K_all_cik_fy = pd.DataFrame(df_10K_all_cik_fy)
    print("CIK-FY All "+np.str(len(df_10K_all_cik_fy)))

    df_10K_cik_missing = df_10K_all_cik_fy[~(df_10K_all_cik_fy["cfy"].isin(df_10K_fnd_cik_fy["cfy"]))]
    print("CIK-FY Missing "+np.str(len(df_10K_cik_missing)))
    
    # GET THE IRRELEVANT TAGS
    df_irl_tags = df_all_tags["tag"][~(df_all_tags["metric"] == df_met_tags4[counter_met])&(df_all_tags["stmt"] == var_FS)]
    df_irl_tags.drop_duplicates(inplace=True)
    df_irl_tags = pd.DataFrame(df_irl_tags)

# GET THE GENERIC TAGS
    df_gnr_tags = df_all_tags["tag_generic"][(df_all_tags["metric"] == df_met_tags4[counter_met])&((df_all_tags["stmt"] == var_FS))]
    df_gnr_tags.drop_duplicates(inplace=True)
    df_gnr_tags = pd.DataFrame(df_gnr_tags)
    df_gnr_tags.rename(columns={"tag_generic":"tag"},inplace=True)

# Found Generic Tag
    df_all_10K_src = df_ANNUAL.copy()
#df_found_generic = df_all_10K[(df_10K_copy["stmt"] == var_IS)&(df_10K_copy["form"] == "10-K")&(df_10K_copy["qtrs"] == "4")&(df_all_10K["cik-fy"].isin(df_10K_cik_missing["cik-fy"]))&(df_all_10K["tag"].str.contains("|".join(df_gnr_tags["tag"])))]
    df_found_generic = df_all_10K_src[(df_all_10K_src["stmt"] == var_FS)
                                  &(df_all_10K_src["qtrs"] == "4")&(df_all_10K_src["cfy"].isin(df_10K_cik_missing["cfy"]))
                                  &(df_all_10K_src["tag"].str.contains("|".join(df_gnr_tags["tag"])))]
#df_found_generic.to_csv((folder_cos+"10K"+var_rel_met+" 2 GENERIC FND.csv"))
#df_10K_fnd_gnr = df_found_generic["cik-fy"]
#df_10K_fnd_gnr.drop_duplicates(inplace=True)
    df_found_generic.drop_duplicates(subset=["tag"],inplace=True)
    print("Found Generic "+np.str(len(df_found_generic)))
    df_found_generic.to_csv((folder_cos+"ANNUAL "+var_FS+" "+df_met_tags4[counter_met]+" 2 GENERIC FND.csv"))
    
    counter_met = counter_met + 1

# Balance Sheet Cash


In [ ]:
# BALANCE SHEET ENTRIES

# Metric of Interest
var_rel_met = "CASH"

var_FS = var_BS

# GET RELEVANT TAGS 
df_rel_tags = df_all_tags["tag"][df_all_tags["metric"] == var_rel_met]
df_rel_tags.drop_duplicates(inplace=True)
df_rel_tags = pd.DataFrame(df_rel_tags)

# Delete Additional Tags
df_addln_tags = pd.DataFrame()
li_cash_tags = ["MARKETABLESECURITIESCURRENT","AVAILABLEFORSALESECURITIESCURRENT","SHORTTERMINVESTMENTS"]
df_addln_tags = df_addln_tags.append(li_cash_tags,ignore_index=True)
df_addln_tags.rename(columns={0:"tag"},inplace=True)
df_rel_tags = df_rel_tags[~(df_rel_tags["tag"].isin(df_addln_tags))]

# GET THE IRRELEVANT TAGS
df_irl_tags = df_all_tags["tag"][~(df_all_tags["metric"] == var_rel_met)]
df_irl_tags.drop_duplicates(inplace=True)
df_irl_tags = pd.DataFrame(df_irl_tags)

# GET THE GENERIC TAGS
df_gnr_tags = df_all_tags["tag_generic"][df_all_tags["metric"] == var_rel_met]
df_gnr_tags.drop_duplicates(inplace=True)
df_gnr_tags = pd.DataFrame(df_gnr_tags)
df_gnr_tags.rename(columns={"tag_generic":"tag"},inplace=True)

# PARE DOWN THE FINANCIAL STATEMENT DATA TO ONLY THOSE TAGS THAT ARE RELEVANT
df_ANNUAL_temp = df_ANNUAL.copy()
df_ANNUAL_rel = df_ANNUAL_temp[(df_ANNUAL_temp["stmt"] == var_FS) &(df_ANNUAL_temp["tag"].isin(df_rel_tags["tag"]))
                               ]

df_ANNUAL_temp = pd.DataFrame()
print(("Number of Relevant Tagged Entries "+np.str(len(df_ANNUAL_rel))))

# RESET THE INDEX ON THE TAG DATAFRAME (# see INDEX_WOES notes)
df_rel_tags2 = df_rel_tags.reset_index(level=0,drop=False)
df_rel_tags2.drop("index",axis=1,inplace=True)
df_rel_tags4 = df_rel_tags2["tag"]

# GET ONLY THE COS AND FY THAT HAVE A RELEVANT (PRIOR) ENTRY
# [Option 1] Pick from a prior file
# df_rel_cos = pd.read_csv((folder_cos+"Found Entries REVENU.csv"),dtype=df_col_types)
#or
# [Option 2] Pick from current set of entries 
df_ANNUAL_temp = df_ANNUAL_rel.copy()
df_rel_cos = df_ANNUAL_temp[["cfy"]]
df_rel_cos.drop_duplicates(subset=["cfy"],inplace=True)
df_ANNUAL_temp = pd.DataFrame()
df_rel_cos_temp = df_rel_cos.copy()
print(("Number of Relevant Co-FY Combos "+np.str(len(df_rel_cos_temp))))

# Create a copy of the relevant entries
df_ANNUAL_rcop = df_ANNUAL_rel.copy()

df_entries_temp = pd.DataFrame()
df_cosfy_tagfound = pd.DataFrame()
df_cosfy_alltagfound = pd.DataFrame()
df_cf_list = pd.DataFrame()
counter_tag = 0

while counter_tag < len(df_rel_tags4):
    
    df_entries_temp = df_ANNUAL_rcop[df_ANNUAL_rcop["cfy"].isin(df_rel_cos_temp["cfy"])]
    
    df_addln_tags = pd.DataFrame()
    li_cash_tags = ["MARKETABLESECURITIESCURRENT","AVAILABLEFORSALESECURITIESCURRENT","SHORTTERMINVESTMENTS"]
    df_addln_tags = df_addln_tags.append(li_cash_tags,ignore_index=True)
    li_current_tag = pd.Series(df_rel_tags4[counter_tag])
    df_addln_tags = df_addln_tags.append(li_current_tag,ignore_index=True)
    df_addln_tags.rename(columns={0:"tag"},inplace=True)
    
    df_cosfy_tagfound = df_entries_temp[(df_entries_temp["tag"].isin(df_addln_tags["tag"]))]
    
    df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)
    
    df_cf_list = df_cosfy_tagfound[["cfy"]]
    df_cf_list.drop_duplicates()
    df_rel_cos_temp = df_rel_cos_temp[~(df_rel_cos_temp["cfy"].isin(df_cf_list["cfy"]))]
    
    print((df_rel_tags4[counter_tag]+ " entries "+np.str(len(df_cosfy_tagfound))))
 
    counter_tag = counter_tag + 1


# GET THE INDUSTRY AND THE SECTOR
df_cik_sic_align = pd.read_csv((folder_mai+"cik_sic_align.csv"),dtype=df_col_types)
df_sic_data = df_cik_sic_align[["sic","Office","Industry Title"]] 
df_sic_data.drop_duplicates(subset="sic",inplace=True)

df_cosfy_alltagfound = pd.merge(df_cosfy_alltagfound,df_currency,left_on="uom",right_on="Currency",how="left")
df_cosfy_alltagfound["value Bn USD"] = df_cosfy_alltagfound["value"] / df_cosfy_alltagfound["inUSD"] / 1000000000

# MERGE THE RELEVANT ENTRIES WITH THE INDUSTRY AND SECTOR
df_10K_met_found = pd.merge(df_cosfy_alltagfound,df_sic_data,on="sic",how="left")

print("Entries Found "+np.str(len(df_10K_met_found)))
df_10K_met_found.to_csv((folder_cos+"ANNUAL "+var_FS+" "+var_rel_met+" 1 FOUND.csv"))

df_10K_fnd_cik_fy = df_10K_met_found["cfy"]
df_10K_fnd_cik_fy.drop_duplicates(inplace=True)
df_10K_fnd_cik_fy = pd.DataFrame(df_10K_fnd_cik_fy)
print("CIK-FY Tag Found "+np.str(len(df_10K_fnd_cik_fy)))

df_10K_all_cik_fy = df_ANNUAL["cfy"]
df_10K_all_cik_fy.drop_duplicates(inplace=True)
df_10K_all_cik_fy = pd.DataFrame(df_10K_all_cik_fy)
print("CIK-FY All "+np.str(len(df_10K_all_cik_fy)))

df_10K_cik_missing = df_10K_all_cik_fy[~(df_10K_all_cik_fy["cfy"].isin(df_10K_fnd_cik_fy["cfy"]))]
print("CIK-FY Missing "+np.str(len(df_10K_cik_missing)))

# Found Generic Tag
df_all_10K_src = df_ANNUAL.copy()
#df_found_generic = df_all_10K[(df_10K_copy["stmt"] == var_IS)&(df_10K_copy["form"] == "10-K")&(df_10K_copy["qtrs"] == "4")&(df_all_10K["cik-fy"].isin(df_10K_cik_missing["cik-fy"]))&(df_all_10K["tag"].str.contains("|".join(df_gnr_tags["tag"])))]
df_found_generic = df_all_10K_src[(df_all_10K_src["stmt"] == var_FS)
                                  &(df_all_10K_src["cfy"].isin(df_10K_cik_missing["cfy"]))
                                  &(df_all_10K_src["tag"].str.contains("|".join(df_gnr_tags["tag"])))]
#df_found_generic.to_csv((folder_cos+"10K"+var_rel_met+" 2 GENERIC FND.csv"))
#df_10K_fnd_gnr = df_found_generic["cik-fy"]
#df_10K_fnd_gnr.drop_duplicates(inplace=True)
df_found_generic.drop_duplicates(subset=["tag"],inplace=True)
print("Found Generic "+np.str(len(df_found_generic)))
df_found_generic.to_csv((folder_cos+"ANNUAL "+var_FS+" "+var_rel_met+" 2 GENERIC FND.csv"))


# Cash Flow Entries

In [ ]:
var_FS = var_CF

# GET METRICs 
df_met_tags = df_all_tags["metric"][(df_all_tags["stmt"] == var_FS)]
df_met_tags.drop_duplicates(inplace=True)
df_met_tags = pd.DataFrame(df_met_tags)

df_met_tags2 = df_met_tags.reset_index(level=0,drop=False)
df_met_tags2.drop("index",axis=1,inplace=True)
df_met_tags4 = df_met_tags2["metric"]

df_all_tags_copy = df_all_tags.copy()
counter_met = 0

while counter_met < len(df_met_tags4):
    
# GET RELEVANT TAGS    
    df_rel_tags = df_all_tags_copy[(df_all_tags_copy["metric"] == df_met_tags4[counter_met])&((df_all_tags_copy["stmt"] == var_FS))]
    df_rel_tags.drop_duplicates(inplace=True)
    df_rel_tags = pd.DataFrame(df_rel_tags)
    print(("Tags in Metric "+df_met_tags4[counter_met]+" are "+np.str(len(df_rel_tags))))

# PARE DOWN THE FINANCIAL STATEMENT DATA TO ONLY THOSE TAGS THAT ARE RELEVANT
    df_ANNUAL_temp = df_ANNUAL.copy()
    df_ANNUAL_rel = df_ANNUAL_temp[(df_ANNUAL_temp["stmt"] == var_FS) &(df_ANNUAL_temp["tag"].isin(df_rel_tags["tag"]))
                               &(df_ANNUAL_temp["qtrs"] == "4")]

    print(("Entries with "+df_met_tags4[counter_met]+" relevant tags are "+np.str(len(df_ANNUAL_rel))))
    
    # RESET THE INDEX ON THE TAG DATAFRAME (# see INDEX_WOES notes)
    df_rel_tags2 = df_rel_tags.reset_index(level=0,drop=False)
    df_rel_tags2.drop("index",axis=1,inplace=True)
    df_rel_tags4 = df_rel_tags2["tag"]
    
    df_ANNUAL_temp = df_ANNUAL_rel.copy()
    df_rel_cos = df_ANNUAL_temp[["cfy"]]
    df_rel_cos.drop_duplicates(subset=["cfy"],inplace=True)
    df_ANNUAL_temp = pd.DataFrame()
    df_rel_cos_temp = df_rel_cos.copy()
    print(("Number of Relevant Co-FY Combos "+np.str(len(df_rel_cos_temp))))
    
    # Create a copy of the relevant entries
    df_ANNUAL_rcop = df_ANNUAL_rel.copy()

    df_entries_temp = pd.DataFrame()
    df_cosfy_tagfound = pd.DataFrame()
    df_cosfy_alltagfound = pd.DataFrame()
    df_cf_list = pd.DataFrame()
    counter_tag = 0
    
    while counter_tag < len(df_rel_tags4):
    
        df_entries_temp = df_ANNUAL_rcop[df_ANNUAL_rcop["cfy"].isin(df_rel_cos_temp["cfy"])]
        df_cosfy_tagfound = df_entries_temp[(df_entries_temp["tag"] == df_rel_tags4[counter_tag])]
    
        df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound,ignore_index=False)
    
        df_cf_list = df_cosfy_tagfound[["cfy"]]
        df_cf_list.drop_duplicates()
#   df_cf_list["cik-fy"] = df_cf_list["cik"] + "-" + df_cf_list["fy"]    
        df_rel_cos_temp = df_rel_cos_temp[~(df_rel_cos_temp["cfy"].isin(df_cf_list["cfy"]))]
    
        print((df_rel_tags4[counter_tag]+ " entries "+np.str(len(df_cosfy_tagfound))))
    
        counter_tag = counter_tag + 1

    df_cosfy_alltagfound = pd.merge(df_cosfy_alltagfound,df_currency,left_on="uom",right_on="Currency",how="left")
    df_cosfy_alltagfound["value Bn USD"] = df_cosfy_alltagfound["value"] / df_cosfy_alltagfound["inUSD"] / 1000000000

    # MERGE THE RELEVANT ENTRIES WITH THE INDUSTRY AND SECTOR
    df_10K_met_found = pd.merge(df_cosfy_alltagfound,df_sic_data,on="sic",how="left")
    
    print("Entries Found "+np.str(len(df_10K_met_found)))
    df_10K_met_found.to_csv((folder_cos+"ANNUAL "+var_FS+" "+df_met_tags4[counter_met]+" 1 FOUND.csv"))

    df_10K_fnd_cik_fy = df_10K_met_found["cfy"]
    df_10K_fnd_cik_fy.drop_duplicates(inplace=True)
    df_10K_fnd_cik_fy = pd.DataFrame(df_10K_fnd_cik_fy)
    print("CIK-FY Tag Found "+np.str(len(df_10K_fnd_cik_fy)))

    df_10K_all_cik_fy = df_ANNUAL["cfy"]
    df_10K_all_cik_fy.drop_duplicates(inplace=True)
    df_10K_all_cik_fy = pd.DataFrame(df_10K_all_cik_fy)
    print("CIK-FY All "+np.str(len(df_10K_all_cik_fy)))

    df_10K_cik_missing = df_10K_all_cik_fy[~(df_10K_all_cik_fy["cfy"].isin(df_10K_fnd_cik_fy["cfy"]))]
    print("CIK-FY Missing "+np.str(len(df_10K_cik_missing)))
    
    # GET THE IRRELEVANT TAGS
    df_irl_tags = df_all_tags["tag"][~(df_all_tags["metric"] == df_met_tags4[counter_met])&(df_all_tags["stmt"] == var_FS)]
    df_irl_tags.drop_duplicates(inplace=True)
    df_irl_tags = pd.DataFrame(df_irl_tags)

# GET THE GENERIC TAGS
    df_gnr_tags = df_all_tags_copy["tag_generic"][(df_all_tags_copy["metric"] == df_met_tags4[counter_met])&((df_all_tags_copy["stmt"] == var_FS))]
    df_gnr_tags.drop_duplicates(inplace=True)
    df_gnr_tags = pd.DataFrame(df_gnr_tags)
    df_gnr_tags.rename(columns={"tag_generic":"tag"},inplace=True)

# Found Generic Tag
    df_all_10K_src = df_ANNUAL.copy()
#df_found_generic = df_all_10K[(df_10K_copy["stmt"] == var_IS)&(df_10K_copy["form"] == "10-K")&(df_10K_copy["qtrs"] == "4")&(df_all_10K["cik-fy"].isin(df_10K_cik_missing["cik-fy"]))&(df_all_10K["tag"].str.contains("|".join(df_gnr_tags["tag"])))]
    df_found_generic = df_all_10K_src[(df_all_10K_src["stmt"] == var_FS)
                                  &(df_all_10K_src["qtrs"] == "4")&(df_all_10K_src["cfy"].isin(df_10K_cik_missing["cfy"]))
                                  &(df_all_10K_src["tag"].str.contains("|".join(df_gnr_tags["tag"])))]
#df_found_generic.to_csv((folder_cos+"10K"+var_rel_met+" 2 GENERIC FND.csv"))
#df_10K_fnd_gnr = df_found_generic["cik-fy"]
#df_10K_fnd_gnr.drop_duplicates(inplace=True)
    df_found_generic.drop_duplicates(subset=["tag"],inplace=True)
    print("Found Generic "+np.str(len(df_found_generic)))
    df_found_generic.to_csv((folder_cos+"ANNUAL "+var_FS+" "+df_met_tags4[counter_met]+" 2 GENERIC FND.csv"))
    
    counter_met = counter_met + 1

# CASH FLOW NET CASH

In [ ]:
var_FS = var_CF
var_rel_met = "NETCASH"

# GET METRICs 
df_met_tags = df_all_tags["metric"][(df_all_tags["stmt"] == var_FS)]
df_met_tags.drop_duplicates(inplace=True)
df_met_tags = pd.DataFrame(df_met_tags)

df_met_tags2 = df_met_tags.reset_index(level=0,drop=False)
df_met_tags2.drop("index",axis=1,inplace=True)
df_met_tags4 = df_met_tags2["metric"]

df_all_tags_copy = df_all_tags.copy()
counter_met = 0
    
# GET RELEVANT TAGS    
df_rel_tags = df_all_tags_copy[(df_all_tags_copy["metric"] == var_rel_met)&((df_all_tags_copy["stmt"] == var_FS))]
df_rel_tags.drop_duplicates(inplace=True)
df_rel_tags = pd.DataFrame(df_rel_tags)
print(("Tags in Metric "+var_rel_met+" are "+np.str(len(df_rel_tags))))

# PARE DOWN THE FINANCIAL STATEMENT DATA TO ONLY THOSE TAGS THAT ARE RELEVANT
df_ANNUAL_temp = df_ANNUAL.copy()
df_ANNUAL_rel = df_ANNUAL_temp[(df_ANNUAL_temp["stmt"] == var_FS) &(df_ANNUAL_temp["tag"].isin(df_rel_tags["tag"]))
                               &(df_ANNUAL_temp["qtrs"] == "4")]

print(("Entries with "+var_rel_met+" relevant tags are "+np.str(len(df_ANNUAL_rel))))
    
    # RESET THE INDEX ON THE TAG DATAFRAME (# see INDEX_WOES notes)
df_rel_tags2 = df_rel_tags.reset_index(level=0,drop=False)
df_rel_tags2.drop("index",axis=1,inplace=True)
df_rel_tags4 = df_rel_tags2["tag"]
    
df_ANNUAL_temp = df_ANNUAL_rel.copy()
df_rel_cos = df_ANNUAL_temp[["cfy"]]
df_rel_cos.drop_duplicates(subset=["cfy"],inplace=True)
df_ANNUAL_temp = pd.DataFrame()
df_rel_cos_temp = df_rel_cos.copy()
print(("Number of Relevant Co-FY Combos "+np.str(len(df_rel_cos_temp))))
    
# Create a copy of the relevant entries
df_cosfy_tagfound = pd.DataFrame()
df_cosfy_alltagfound = pd.DataFrame()
df_cf_list = pd.DataFrame()
counter_tag = 0
df_ANNUAL_rcop = df_ANNUAL_rel.copy()
df_ANNUAL_rel.sort_values(by="cfy",ascending=False,inplace=True)
df_rel_tags4 = pd.DataFrame()    
df_rel_tags4 = pd.Series(["NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIES","NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIESCONTINUINGOPERATIONS"])
df_rel_tags4 = pd.DataFrame(df_rel_tags4)
df_rel_tags4.rename(columns={0:"tag"},inplace=True)  
#df_entries_temp = df_ANNUAL_rcop[df_ANNUAL_rcop["cik-cfy"].isin(df_rel_cos_temp["cik-cfy"])]
df_cosfy_tagfound = df_ANNUAL_rcop[(df_ANNUAL_rcop["tag"].isin(df_rel_tags4["tag"]))&df_ANNUAL_rcop["cfy"].isin(df_rel_cos_temp["cfy"])]

df_cosfy_tagfound_copy = df_cosfy_tagfound.copy()

df_cosfy_tagfound_copy.sort_values(by=["cfy","value"],inplace=True)
df_cosfy_tagfound_copy.drop_duplicates(subset="cfy",inplace=True)

df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound_copy,ignore_index=False)
print((var_rel_met+ " OPRT entries "+np.str(len(df_cosfy_tagfound))))

df_ANNUAL_rcop = df_ANNUAL_rel.copy()
df_rel_tags4 = pd.DataFrame()
df_rel_tags4 = pd.Series(["NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIES","NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIESCONTINUINGOPERATIONS"])
df_rel_tags4 = pd.DataFrame(df_rel_tags4)
df_rel_tags4.rename(columns={0:"tag"},inplace=True)  
df_cosfy_tagfound = df_ANNUAL_rcop[(df_ANNUAL_rcop["tag"].isin(df_rel_tags4["tag"]))&df_ANNUAL_rcop["cfy"].isin(df_rel_cos_temp["cfy"])]
df_cosfy_tagfound_copy = df_cosfy_tagfound.copy()

df_cosfy_tagfound_copy.sort_values(by=["cfy","value"],inplace=True)
df_cosfy_tagfound_copy.drop_duplicates(subset="cfy",inplace=True)

df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound_copy,ignore_index=False)
print((var_rel_met+ " INVST entries "+np.str(len(df_cosfy_tagfound))))

df_ANNUAL_rcop = df_ANNUAL_rel.copy()
df_rel_tags4 = pd.DataFrame()
df_rel_tags4 = pd.Series(["NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIES","NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIESCONTINUINGOPERATIONS"])
df_rel_tags4 = pd.DataFrame(df_rel_tags4)
df_rel_tags4.rename(columns={0:"tag"},inplace=True)  
df_cosfy_tagfound = df_ANNUAL_rcop[(df_ANNUAL_rcop["tag"].isin(df_rel_tags4["tag"]))&df_ANNUAL_rcop["cfy"].isin(df_rel_cos_temp["cfy"])]
df_cosfy_tagfound_copy = df_cosfy_tagfound.copy()

df_cosfy_tagfound_copy.sort_values(by=["cfy","value"],inplace=True)
df_cosfy_tagfound_copy.drop_duplicates(subset="cfy",inplace=True)

df_cosfy_alltagfound = df_cosfy_alltagfound.append(df_cosfy_tagfound_copy,ignore_index=False)
print((var_rel_met+ " FINANC entries "+np.str(len(df_cosfy_tagfound))))

df_cosfy_alltagfound = pd.merge(df_cosfy_alltagfound,df_currency,left_on="uom",right_on="Currency",how="left")
df_cosfy_alltagfound["value Bn USD"] = df_cosfy_alltagfound["value"] / df_cosfy_alltagfound["inUSD"] / 1000000000

# MERGE THE RELEVANT ENTRIES WITH THE INDUSTRY AND SECTOR
df_10K_met_found = pd.merge(df_cosfy_alltagfound,df_sic_data,on="sic",how="left")
    
print("Entries Found "+np.str(len(df_10K_met_found)))
df_10K_met_found.to_csv((folder_cos+"ANNUAL "+var_FS+" "+var_rel_met+" 1 FOUND.csv"))

df_10K_fnd_cik_fy = df_10K_met_found["cfy"]
df_10K_fnd_cik_fy.drop_duplicates(inplace=True)
df_10K_fnd_cik_fy = pd.DataFrame(df_10K_fnd_cik_fy)
print("CIK-FY Tag Found "+np.str(len(df_10K_fnd_cik_fy)))

df_10K_all_cik_fy = df_ANNUAL["cfy"]
df_10K_all_cik_fy.drop_duplicates(inplace=True)
df_10K_all_cik_fy = pd.DataFrame(df_10K_all_cik_fy)
print("CIK-FY All "+np.str(len(df_10K_all_cik_fy)))

df_10K_cik_missing = df_10K_all_cik_fy[~(df_10K_all_cik_fy["cfy"].isin(df_10K_fnd_cik_fy["cfy"]))]
print("CIK-FY Missing "+np.str(len(df_10K_cik_missing)))
    
    # GET THE IRRELEVANT TAGS
df_irl_tags = df_all_tags["tag"][~(df_all_tags["metric"] == var_rel_met)&(df_all_tags["stmt"] == var_FS)]
df_irl_tags.drop_duplicates(inplace=True)
df_irl_tags = pd.DataFrame(df_irl_tags)

# GET THE GENERIC TAGS
df_gnr_tags = df_all_tags["tag_generic"][(df_all_tags["metric"] == var_rel_met)&((df_all_tags["stmt"] == var_FS))]
df_gnr_tags.drop_duplicates(inplace=True)
df_gnr_tags = pd.DataFrame(df_gnr_tags)
df_gnr_tags.rename(columns={"tag_generic":"tag"},inplace=True)

# Found Generic Tag
df_all_10K_src = df_ANNUAL.copy()
#df_found_generic = df_all_10K[(df_10K_copy["stmt"] == var_IS)&(df_10K_copy["form"] == "10-K")&(df_10K_copy["qtrs"] == "4")&(df_all_10K["cik-fy"].isin(df_10K_cik_missing["cik-fy"]))&(df_all_10K["tag"].str.contains("|".join(df_gnr_tags["tag"])))]
df_found_generic = df_all_10K_src[(df_all_10K_src["stmt"] == var_FS)
                                  &(df_all_10K_src["qtrs"] == "4")&(df_all_10K_src["cfy"].isin(df_10K_cik_missing["cfy"]))
                                  &(df_all_10K_src["tag"].str.contains("|".join(df_gnr_tags["tag"])))]
#df_found_generic.to_csv((folder_cos+"10K"+var_rel_met+" 2 GENERIC FND.csv"))
#df_10K_fnd_gnr = df_found_generic["cik-fy"]
#df_10K_fnd_gnr.drop_duplicates(inplace=True)
df_found_generic.drop_duplicates(subset=["tag"],inplace=True)
print("Found Generic "+np.str(len(df_found_generic)))
df_found_generic.to_csv((folder_cos+"ANNUAL "+var_FS+" "+var_rel_met+" 2 GENERIC FND.csv"))
    
